In [98]:
import os
import csv
import copy
import pprint
import xml.etree.ElementTree as ET

In [39]:
clip_dict = {}
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/topics/ins.action.topics", "r") as f:
    lines = f.readlines()
    for l in lines:
        label, clip = l.split()
        clip_dict[clip] = label

In [40]:
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/query/person_labels.csv", "r") as f:
    reader = csv.reader(f, delimiter=" ")
    person_label_dict = {r[0]: r[1] for r in reader}

In [41]:
tree = ET.parse("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/topics/ins.auto.topics.2019.xml")
root = tree.getroot()
action_dict = {}
for child in root:
    action = child.attrib["action"]
    person = child.attrib["person"]
    if action in action_dict:
        action_dict[action].append(person.lower())
    else:
        action_dict[action] = [person.lower()]

In [42]:
person_idx_dict = {}
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/query/person_features.csv", "r") as f:
    reader = csv.reader(f, delimiter=" ")
    for r in reader:
        person = os.path.basename(r[0]).split(".")[0]
        if person in person_idx_dict:
            person_idx_dict[person].append(int(r[1]))
        else:
            person_idx_dict[person] = [int(r[1])]

In [48]:
features = []
cnt = 1
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/query/features_csv.csv", "r") as f:
    reader = csv.reader(f, delimiter=" ")
    for r in reader:
        tmp = os.path.basename(r[5]).split("_")
        clip = tmp[0] + "_" + tmp[1]
        action = clip_dict[clip]
        if action not in action_dict:
            continue
        for person in action_dict[action]:
            indexes = person_idx_dict[person]
            for idx in indexes:
                q = copy.deepcopy(r)
                q[0] = cnt
                q[4] = idx
                q[7] = person_label_dict[person]
                features.append(q)
                cnt += 1

In [50]:
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/query/features.csv", "w") as f:
    writer = csv.writer(f, delimiter=" ")
    writer.writerows(features)

In [49]:
features

[[1,
  '2',
  '1',
  '0',
  28,
  '/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/detected_query_videos/clip1_011_id_8.mp4',
  '1',
  '8'],
 [2,
  '2',
  '1',
  '0',
  29,
  '/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/detected_query_videos/clip1_011_id_8.mp4',
  '1',
  '8'],
 [3,
  '2',
  '1',
  '0',
  30,
  '/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/detected_query_videos/clip1_011_id_8.mp4',
  '1',
  '8'],
 [4,
  '2',
  '1',
  '0',
  31,
  '/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/detected_query_videos/clip1_011_id_8.mp4',
  '1',
  '8'],
 [5,
  '2',
  '1',
  '0',
  16,
  '/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/detected_query_videos/clip1_011_id_8.mp4',
  '1',
  '5'],
 [6,
  '2',
  '1',
  '0',
  17,
  '/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/detected_query_videos/clip1_011_id_8.mp4',
 

In [51]:
import numpy as np

In [61]:
person_fvs = np.load("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/gallery/person_features.npy")

In [75]:
np.sum(person_fvs[1000] ** 2)

0.9948073

In [85]:
action_fvs = np.load("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/gallery/action_features.npy")
new_action_fvs = action_fvs / np.array([np.sqrt(np.sum(action_fvs ** 2, axis=1))]).T
np.save("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/gallery/action_features.npy", new_action_fvs)

In [72]:
new_action_fvs = action_fvs / np.array([np.sqrt(np.sum(action_fvs ** 2, axis=1))]).T

In [84]:
np.sum(new_action_fvs[0] ** 2)

1.0

In [86]:
new_action_fvs.shape

(263490, 2304)

In [104]:
action_rankings = np.load("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/action_rankings.npy")

In [91]:
action_rankings

array([[     1,  35601,  74670, ...,  97799,  80161,  49620],
       [     2, 140639,   1348, ..., 115169,  88709, 117712],
       [     3,  99441,  67707, ..., 109521,  79218,  44145],
       ...,
       [   342, 123195, 129515, ...,  79218, 119708, 117712],
       [   343, 123195, 129515, ...,  61627, 143173,  10282],
       [   344, 105867, 129515, ...,  52734,  27842,  79218]])

In [105]:
action_rankings

array([[     1,  55580,  35601, ...,  97799,  35869,  49620],
       [     2,  47800,   1348, ..., 109521, 117712,  26084],
       [     3,  97283,  67707, ..., 115169, 109521, 117712],
       ...,
       [   342, 129515, 123195, ...,  78316, 119708, 117712],
       [   343, 129515, 105867, ..., 142079,  54932,  68163],
       [   344, 129515,  94072, ...,  27842,  79218, 126306]])

In [106]:
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/query/features.csv", "r") as f:
    reader = csv.reader(f, delimiter=" ")
    query_dict = {int(idx): [path, label] for idx, _, _, _, _, path, label, _ in reader}
    
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/gallery/features.csv", "r") as f:
    reader = csv.reader(f, delimiter=" ")
    gallery_dict = {int(idx): [path, label] for idx, _, _, _, _, path, label, _ in reader}

In [107]:
query_idx, gallery_indexes = action_rankings[1][0], action_rankings[1][1:]
print(query_dict[query_idx])
results = []
for idx in gallery_indexes:
    if gallery_dict[idx] in results:
        continue
    else:
        results.append(gallery_dict[idx])
pprint.pprint(results[:10])

['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/detected_query_videos/clip1_011_id_8.mp4', '1']
[['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot26_204_id_274184.mp4',
  '99'],
 ['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot155_838_id_1656133.mp4',
  '99'],
 ['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot72_1501_id_811375.mp4',
  '99'],
 ['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot135_695_id_1467733.mp4',
  '99'],
 ['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot126_206_id_1340116.mp4',
  '99'],
 ['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot169_1390_id_1808241.mp4',
  '99'],
 ['/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot61_1536_id_691484.mp4',
  '99'],
 ['/net/per610a/export/das18a/satoh-lab/shar

In [101]:
set(results)

TypeError: unhashable type: 'list'